In [1]:
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv()) # read .env file
apikey = os.getenv('NVIDIA_API_KEY')

# A Gentle Introduction to RAG Applications

This notebook creates a simple RAG (Retrieval-Augmented Generation) system to answer questions from a PDF document using an open-source model.

In [2]:
# download BOT 2566 annual report from https://www.bot.or.th/content/dam/bot/documents/th/thai-economy/state-of-thai-economy/annual-report/annual-econ-report-th-2566.pdf

PDF_FILE = "llm/labs/LangChain/annual-econ-report-th-2566.pdf"

In [3]:
!ls -l llm/labs/LangChain/annual-econ-report-th-2566.pdf

-rw-r--r-- 1 gable-test gable-test 342845 Aug 18 03:29 llm/labs/LangChain/annual-econ-report-th-2566.pdf


## Loading the PDF document

Let's start by loading the PDF document and breaking it down into separate pages.

<img src='images/documents.png' width="1000">

In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(PDF_FILE)
pages = loader.load()

print(f"Number of pages: {len(pages)}")
print(f"Length of a page: {len(pages[1].page_content)}")
print("Content of a page:", pages[1].page_content)

Number of pages: 7
Length of a page: 2988
Content of a page:  
 
1 
 สรุปภาวะเศรษฐกิจการเงินต่างประเทศและไทย ปี 2566 และแนวโน้ม  
เศรษฐกิจต่างประเทศปี 2566 และแนวโน้ม  
เศรษฐกิจโลกปี 256 6 ขยายตัวในอัตราที่ชะลอลงจากปีก่อน โดยการบริโภคภาคเอกชน  
โดยเฉพาะภาคบริการ ยังเป็นแรงขับเคลื่อนเศรษฐกิจที่สำคัญในหลายประเทศ จากการผ่อนคลายมาตรการ
ควบคุมการเดินทางระหว่างประเทศ และนโยบายการเปิดประเทศของจีน ในช่วงต้นปี  ขณะที่ปริมาณการค้า
โลกที่ชะลอตัว ส่งผลให้การผลิตภาคอุตสาหกรรมและการส่งออกสินค้าใน หลายประเทศ โดยเฉพาะกลุ่ม
ประเทศเอเชีย ชะลอลง ซึ่งเป็นผลจาก  (1) การดำเนินนโยบายการเงินที่เข้มงวดของธนาคารกลาง หลาย
ประเทศเพื่อดูแลเงินเฟ้อที่มีแนวโน้มทรงตัวในระดับสูงนานกว่าคาด ซึ่งส่งผลให้สถาบันการเงินบางแห่งใน
สหรัฐฯ และยุโรปประสบปัญหาด้านสภาพคล่อง ในไตรมาส 1 อย่างไรก็ดี ผลต่อเศรษฐกิจและการเงินโลก
ค่อนข้างจำกัด เนื่องจากทางการได้ดำเนินมาตรการสกัดผลกระทบที่จะลุกลามเป็นวงกว้างได้อย่างรวดเร็ว 
อาทิ การค้ำประกันเงินฝาก และการตั้งกองทุนเพื่อเสริมสภาพคล่องแก่สถาบันการเงิน เป็นต้น  (2) การฟื้นตัว
ของเศรษฐกิจจีนท

## Splitting the pages in chunks

Pages are too long, so let's split pages into different chunks.

<img src='images/splitter.png' width="1000">


In [56]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = splitter.split_documents(pages)
print(f"Number of chunks: {len(chunks)}")
print(f"Length of a chunk: {len(chunks[1].page_content)}")
print("Content of a chunk:", chunks[1].page_content)


Number of chunks: 24
Length of a chunk: 919
Content of a chunk: 1 
 สรุปภาวะเศรษฐกิจการเงินต่างประเทศและไทย ปี 2566 และแนวโน้ม  
เศรษฐกิจต่างประเทศปี 2566 และแนวโน้ม  
เศรษฐกิจโลกปี 256 6 ขยายตัวในอัตราที่ชะลอลงจากปีก่อน โดยการบริโภคภาคเอกชน  
โดยเฉพาะภาคบริการ ยังเป็นแรงขับเคลื่อนเศรษฐกิจที่สำคัญในหลายประเทศ จากการผ่อนคลายมาตรการ
ควบคุมการเดินทางระหว่างประเทศ และนโยบายการเปิดประเทศของจีน ในช่วงต้นปี  ขณะที่ปริมาณการค้า
โลกที่ชะลอตัว ส่งผลให้การผลิตภาคอุตสาหกรรมและการส่งออกสินค้าใน หลายประเทศ โดยเฉพาะกลุ่ม
ประเทศเอเชีย ชะลอลง ซึ่งเป็นผลจาก  (1) การดำเนินนโยบายการเงินที่เข้มงวดของธนาคารกลาง หลาย
ประเทศเพื่อดูแลเงินเฟ้อที่มีแนวโน้มทรงตัวในระดับสูงนานกว่าคาด ซึ่งส่งผลให้สถาบันการเงินบางแห่งใน
สหรัฐฯ และยุโรปประสบปัญหาด้านสภาพคล่อง ในไตรมาส 1 อย่างไรก็ดี ผลต่อเศรษฐกิจและการเงินโลก
ค่อนข้างจำกัด เนื่องจากทางการได้ดำเนินมาตรการสกัดผลกระทบที่จะลุกลามเป็นวงกว้างได้อย่างรวดเร็ว 
อาทิ การค้ำประกันเงินฝาก และการตั้งกองทุนเพื่อเสริมสภาพคล่องแก่สถาบันการเงิน เป็นต้น  (2) การฟื้นตัว


## Storing the chunks in a vector store

We can now generate embeddings for every chunk and store them in a vector store.

<img src='images/vectorstore.png' width="1000">


In [57]:
from langchain_community.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embeddings = NVIDIAEmbeddings(
    model="nvidia/nv-embedqa-e5-v5",
    base_url="http://10.134.1.48:31384/v1",
    api_key=apikey,
    truncate="NONE"
)

vectorstore = FAISS.from_documents(chunks, embeddings)

## Setting up a retriever

We can use a retriever to find chunks in the vector store that are similar to a supplied question.

<img src='images/retriever.png' width="1000">



In [58]:
retriever = vectorstore.as_retriever()
retriever.invoke("แนวโน้มการขยายตัวทางเศรษฐกิจของปี 2566 เป็นอย่างไรเมื่อเทียบกับปีก่อน")

[Document(metadata={'source': 'llm/labs/LangChain/annual-econ-report-th-2566.pdf', 'page': 5}, page_content='5 \n  \nรวมเหตุการณ์สำคัญที่กระทบเศรษฐกิจการเงินโลกและไทย  \n \n \n \n \n \n \n \n \n \nปี 2566 เป็นอีกปีที่มีเหตุการณ์สำคัญ ที่ส่งผลกระทบต่อระบบเศรษฐกิจการเงินของโลกและไทย ดังนี้  \n1. จีนได้ประกาศเปิดประเทศในวันที่ 8 ม.ค. 2566 โดยอนุญาตให้ชาวจีนเดินทางออกนอกประเทศได้\nโดยไม่ต้องตรวจเชื้อโควิดและกักตัว หลังจากนั้นในวันที่ 15 มี.ค. 2566 จีนประกาศอนุญาตให้\nนักท่องเที่ยวต่างชาติเดินทางกลับไปเที่ยวจีนได้  \n2. วันที่ 12 มี.ค. 2566 ทางการสหรัฐประกาศปิดกิจการ  Signature Bank หลังจากนั้นในวันที่ 17 มี .ค. \n2566 ธนาคาร Silicon Valley (SVB) ประกาศล้มละลาย  นอกจากนี้  ในวันที่ 19 มี.ค. 2566 Union \nBank of Switzerland  (UBS) ซื้อกิจการของ Credit Suis se ในราคากว่า 3 พันล้านฟรังก์ โดยได้รับ\nความสนับสนุนจากธนาคารกลาง และรัฐบาลสวิตเซอร์แลนด์  \n3. การเลือกตั้งสมาชิกสภาผู้แทนราษฎรไทยเป็นการทั่วไปครั้งที่ 27 จัดขึ้นเมื่อวันที่ 14 พฤษภาคม 2566  \nและมีการแถลงข่าวการจัดตั้งรัฐบาล ในวันที่ 21

## Configuring the model

We'll be using Ollama to load the local model in memory. After creating the model, we can invoke it with a question to get the response back.

<img src='images/model.png' width="1000">

In [59]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# llm model
model = ChatNVIDIA(base_url="http://10.134.1.48:30644/v1", 
                   model="meta/llama3-8b-instruct", 
                   temperature=0.0, max_tokens=2000)  # Increase number of max_tokens for Thai

model.invoke("อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566")

AIMessage(content="Based on various sources, including market research reports and industry trends, here are some industries that are expected to grow in 2023 (2566):\n\n1. **E-commerce and Digital Payments**: The COVID-19 pandemic has accelerated the shift to online shopping, and this trend is expected to continue. E-commerce sales are expected to grow by 15-20% in 2023.\n2. **Healthcare Technology**: The healthcare industry is expected to invest more in technology, including telemedicine, artificial intelligence, and data analytics, to improve patient care and reduce costs.\n3. **Renewable Energy**: As governments around the world set ambitious targets to reduce carbon emissions, the demand for renewable energy sources such as solar and wind power is expected to increase.\n4. **Cybersecurity**: With the rise of online transactions and data breaches, cybersecurity is becoming a major concern for businesses and individuals. The demand for cybersecurity solutions and services is expecte

## Parsing the model's response

The response from the model is an `AIMessage` instance containing the answer. We can extract the text answer by using the appropriate output parser. We can connect the model and the parser using a chain.

<img src='images/parser.png' width="1000">

In [60]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
print(chain.invoke("อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566"))

Based on various sources, including market research reports and industry trends, here are some industries that are expected to grow in 2023 (2566):

1. **E-commerce and Digital Payments**: The COVID-19 pandemic has accelerated the shift to online shopping, and this trend is expected to continue. E-commerce sales are expected to grow by 15-20% in 2023.
2. **Healthcare Technology**: The healthcare industry is expected to invest more in technology, including telemedicine, artificial intelligence, and data analytics, to improve patient care and reduce costs.
3. **Renewable Energy**: As governments around the world set ambitious targets to reduce carbon emissions, the demand for renewable energy sources such as solar and wind power is expected to increase.
4. **Cybersecurity**: With the rise of online transactions and data breaches, cybersecurity is becoming a major concern for businesses and individuals. The demand for cybersecurity solutions and services is expected to grow by 10-15% in 2

## Setting up a prompt

In addition to the question we want to ask, we also want to provide the model with the context from the PDF file. We can use a prompt template to define and reuse the prompt we'll use with the model.

<img src='images/prompt.png' width="1000">

In [61]:
from langchain.prompts import PromptTemplate

template = """
You are an assistant that provides answers to questions based on
a given context. 

Alwasy answer the question based on the context in Thai. If you can't answer the
question, reply "ฉันมีข้อมูลไม่เพียงพอ".

Be as concise as possible and go straight to the point.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="นี่คือ context", question="นีคือคำถาม"))


You are an assistant that provides answers to questions based on
a given context. 

Alwasy answer the question based on the context in Thai. If you can't answer the
question, reply "ฉันมีข้อมูลไม่เพียงพอ".

Be as concise as possible and go straight to the point.

Context: นี่คือ context

Question: นีคือคำถาม



## Adding the prompt to the chain

We can now chain the prompt with the model and the parser.

<img src='images/chain1.png' width="1000">

In [62]:
chain = prompt | model | parser

chain.invoke({
    "context": "Anna's sister is Susan", 
    "question": "Who is Susan's sister?"
})


'แอนนา'

## Adding the retriever to the chain

Finally, we can connect the retriever to the chain to get the context from the vector store.

<img src='images/chain2.png' width="1000">

In [63]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

## Using the chain to answer questions

Finally, we can use the chain to ask questions that will be answered using the PDF document.

In [70]:
questions = [
    "อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566",
    "จงสรุปภาพรวมด้านอัตราเงินเฟ้อและเศรษฐกิจภาคครัวเรือน",
    "กลุ่มสินค้าและบริการอะไรที่ไทยควรมีการพัฒนาให้สามารถแข่งขันในตลาดโลกได้",
    "แนวโน้มเศรษฐกิจไทยในอนาคตจะเป็นอย่างไร",
    "ความเสี่ยงในอนาคตที่ไทยควรเตรียมรับมือ",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print("*************************\n")

Question: อุตสาหกรรมที่คาดว่าจะมีการขยายตัวในปี 2566
Answer: บริการภาคบริการ
*************************

Question: จงสรุปภาพรวมด้านอัตราเงินเฟ้อและเศรษฐกิจภาคครัวเรือน
Answer: อัตราเงินเฟ้อในปี 2566 อยู่ในระดับต่ำ โดยดัชนีราคาผู้บริโภคพื้นฐาน (ไม่รวมราคาอาหารสดและพลังงาน) อยู่ที่ 1.87% และอัตราการว่างงานอยู่ที่ 1.1% ส่วนหนี้สาธารณะ ณ สิ้นงวดอยู่ที่ 10,928 พันล้านบาท สัดส่วนหนี้สาธารณะต่อ GDP อยู่ที่ 61.7%
*************************

Question: กลุ่มสินค้าและบริการอะไรที่ไทยควรมีการพัฒนาให้สามารถแข่งขันในตลาดโลกได้
Answer: ตามรายงานภาวะเศรษฐกิจไทย ปี 2566 กลุ่มสินค้าและบริการที่ไทยควรมีการพัฒนาให้สามารถแข่งขันในตลาดโลกได้แก่ สินค้าเกษตรและอาหารสินค้าขั้นปลาย รวมทั้งสินค้าอุตสาหกรรม เช่น สินค้าเหล็ก สินค้าพลาสติก สินค้าเครื่องมือเครื่องใช้ และบริการทางการเงิน รวมทั้งบริการด้านเทคโนโลยีสารสนเทศ
*************************

Question: แนวโน้มเศรษฐกิจไทยในอนาคตจะเป็นอย่างไร
Answer: แนวโน้มเศรษฐกิจไทยในอนาคตมีความไม่แน่นอนและผันผวน เนื่องจากปัจจัยภายนอกที่มีผลกระทบต่อเศรษฐกิจไทย เช่น เศรษฐกิจและตล